# Intelligent AI ChatBot in Python (NeuralNine)

In [ ]:
# Intelligent AI ChatBot in Python
# this is going to dyamically understand and respond to greetings
# but the responses will be static

# our files will be : intents.json, responses.json

## TRAINING CHATBOT

In [35]:
# imports
import random
import json
import pickle   # for serialization
import numpy as np
import sqlite3

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

# lemmatizing our words
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### *loading our intents*

In [2]:
# loading file
from google.colab import files
uploaded = files.upload()

Saving intents_eng.json to intents_eng.json


In [3]:
print(uploaded)

{'intents_eng.json': b'{\r\n    "intents": [\r\n        {\r\n            "tag": "greetings",\r\n            "pattern": ["hello", "hey", "hi", "good day", "good morning"],\r\n            "responses": ["Hello", "What can I do for you?"]\r\n        },  \r\n        {\r\n            "tag": "goodbye",\r\n            "pattern": ["cya", "see you later", "I\'m leaving", "goodbye", "have a good day", "I am leaving", "bye", "see ya"],\r\n            "responses": ["Goodbye", "Nice to have met you"]\r\n        },  \r\n        {\r\n            "tag": "age",\r\n            "pattern": ["how old", "how old is florian", "what is your age", "how old are you?", "age?"],\r\n            "responses": ["My owner is 21 years old!", "21 years!", "21"]\r\n        },  \r\n        {\r\n            "tag": "name",\r\n            "pattern": ["What is your name", "what should I call you?", "what is your name", "who are you", "can you tell me your name"],\r\n            "responses": ["You can call me Neural!", "I\'m ne

In [4]:
intents =  json.loads(uploaded['intents_eng.json'])
print(intents['intents'][3]['tag'])

name


### *Creating intermediate Data*

In [5]:
# creatig useful lists
words = []
classes = []
docs = []

ignore_letters = ["!", ".", ",", "?"]

# iterating over intents
for intent in intents['intents']:
  for pat in intent['pattern']:
    # tokenizing and adding tokens to the word list
    word_list = nltk.word_tokenize(pat)
    if word_list is not None:
      words.extend(word_list)

      # appending 
      docs.append((word_list, intent['tag']))

      if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [6]:
print(words)

['hello', 'hey', 'hi', 'good', 'day', 'good', 'morning', 'cya', 'see', 'you', 'later', 'I', "'m", 'leaving', 'goodbye', 'have', 'a', 'good', 'day', 'I', 'am', 'leaving', 'bye', 'see', 'ya', 'how', 'old', 'how', 'old', 'is', 'florian', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', '?', 'age', '?', 'What', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', '?', 'what', 'is', 'your', 'name', 'who', 'are', 'you', 'can', 'you', 'tell', 'me', 'your', 'name', 'I', "'d", 'like', 'to', 'buy', 'something', 'What', 'are', 'your', 'products', '?', 'what', 'do', 'you', 'recommend', '?', 'What', 'are', 'you', 'selling', '?', 'When', 'are', 'you', 'guys', 'open', '?', 'What', 'are', 'your', 'hours', 'hours', 'of', 'operation', 'price', 'value', 'mrp', 'What', 'is', 'the', 'price', 'of', 'maize', 'How', 'much', 'is', 'Wheat', 'worth', 'selling', 'price', 'of', 'rice', 'what', 'stocks', 'do', 'I', 'own', '?', 'how', 'are', 'my', 'shares', '?', 'what', 'companies', 'am', 'I', 'invest

### *Lemmatizing*

In [7]:
# lemmatizing our words
lemmatizer = WordNetLemmatizer()
  
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

# removing duplicates
words = sorted(set(words))
classes = sorted(set(classes))

In [8]:
print(words)

["'d", "'m", 'How', 'I', 'What', 'Wheat', 'When', 'a', 'age', 'am', 'are', 'buy', 'bye', 'call', 'can', 'company', 'cya', 'day', 'do', 'doing', 'florian', 'good', 'goodbye', 'guy', 'have', 'hello', 'hey', 'hi', 'hour', 'how', 'in', 'invested', 'is', 'later', 'leaving', 'like', 'maize', 'me', 'morning', 'mrp', 'much', 'my', 'name', 'of', 'old', 'open', 'operation', 'own', 'price', 'product', 'recommend', 'rice', 'see', 'selling', 'share', 'should', 'something', 'stock', 'tell', 'the', 'to', 'value', 'what', 'who', 'worth', 'ya', 'you', 'your']


### *Saving Data*

In [9]:
# adding all info to file
pickle.dump(words, open("words.pkl", 'wb'))
pickle.dump(classes, open("classes.pkl", 'wb'))
pickle.dump(docs, open("docs.pkl", 'wb'))

### *Creattig Training Data*

In [10]:
# representing words as numeric data using bag of words
# so that we could use this in a neural network
training = []
output_empty = [0]*len(classes)

for doc in docs:
  bag = []
  word_patterns = doc[0]
  word_patterns = [lemmatizer.lemmatize(word) for word in word_patterns]

  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  # print(output_row)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

### *Pre-Processing*

In [11]:
# shufling our data
random.shuffle(training)
training = np.array(training)

print(training.shape)


# spliting to X and y values
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# print(np.asarray(train_x))
print(train_y.shape)

(40, 2)
(40, 8)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


### *Building Neural Net*

In [12]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

In [13]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.09, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               8832      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 17,608
Trainable params: 17,608
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(train_x, train_y, epochs=200, batch_size=4, verbose=1)

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.1096 - accuracy: 0.1000
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1563 - accuracy: 0.0500
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1197 - accuracy: 0.0750
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0426 - accuracy: 0.2000
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1328 - accuracy: 0.1250
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0523 - accuracy: 0.1750
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0421 - accuracy: 0.2000
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0256 - accuracy: 0.2750
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0400 - accuracy: 0.3000
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1319 - accuracy: 0.2000
Epoch 11/

In [15]:
model.save("ai_chatbot.h5", history)

## EXECUTING CHATBOT

### *Loading Our Crop Data*

In [16]:
# checking existing intents file
print(intents['intents'][3]['tag'])

name


In [17]:
# loading file
from google.colab import files
uploaded = files.upload()

Saving crop_data_eng.json to crop_data_eng.json


In [18]:
print(uploaded)

{'crop_data_eng.json': b'{\r\n    "Bhindi(Ladies Finger)": {\r\n        "State": "Bihar",\r\n        "District": "Kaimur/Bhabhua",\r\n        "Market": "Kaimur",\r\n        "Commodity": "Bhindi(Ladies Finger)",\r\n        "Variety": "Bhindi",\r\n        "Arrival_Date": "08/06/2022",\r\n        "Min_Price": "1200",\r\n        "Max_Price": "1300",\r\n        "Modal_Price": "1250"\r\n    },\r\n    "Green Chilli": {\r\n        "State": "Bihar",\r\n        "District": "Kaimur/Bhabhua",\r\n        "Market": "Kaimur",\r\n        "Commodity": "Green Chilli",\r\n        "Variety": "Green Chilly",\r\n        "Arrival_Date": "08/06/2022",\r\n        "Min_Price": "2200",\r\n        "Max_Price": "2400",\r\n        "Modal_Price": "2300"\r\n    },\r\n    "Onion": {\r\n        "State": "Bihar",\r\n        "District": "Kaimur/Bhabhua",\r\n        "Market": "Kaimur",\r\n        "Commodity": "Onion",\r\n        "Variety": "1st Sort",\r\n        "Arrival_Date": "08/06/2022",\r\n        "Min_Price": "1200"

In [22]:
crop_data =  json.loads(uploaded['crop_data_eng.json'])
print(crop_data["Mustard"]["Min_Price"])

6100


### *Loading up our saved files*

In [25]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
docs = pickle.load(open('docs.pkl', 'rb'))

model = load_model("ai_chatbot.h5")

### *Creating Crop Data Data Structure by Querying dataset*

In [26]:
crop_names = list(crop_data.keys())

### *Creating Helper Functions*



In [37]:
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)

  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  
  return np.array(bag)

def predict_class(sentence):
  ERROR_THRESHOLD = 0.25

  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]

  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)

  return_list = []
  for r in results:
    return_list.append({ 'intent': classes[r[0]], 'probability': str(r[1]) })

  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

def get_crop_data(msg):
  bow = list(msg.split())
  # bow = [string.lower() for string in bow]
  bow = [string for string in bow]
  print("message words : ", bow)
  print("crops : ", crop_names)

  # common_crops = []
  # for sub in bow:
  #   common_crops += list(s for s in crop_names if sub.lower() in s.lower())

  common_crops = list(set(bow) & set(crop_names))
  print("common crops : ", common_crops)

  if not common_crops:
    return "No Crop Data Found, Please Try again"
  else:
    crop = crop_data[common_crops[0]]

    crop_res = "Here is the relevent information on the {} crop: \n".format(common_crops[0])
    crop_res += "Commodity : {}\n".format(crop["Commodity"])
    crop_res += "Variety : {}\n".format(crop["Variety"])
    crop_res += "Area : {}, {}, {}\n".format(crop["Market"], crop["District"], crop["State"])
    crop_res += "Maximum Price : {}\n".format(crop["Max_Price"])
    crop_res += "Average Price : {}\n".format(crop["Modal_Price"])
    crop_res += "Minimum Price : {}\n".format(crop["Min_Price"])
    return crop_res


## Running BOT

In [38]:
while True:
  message = input('Input Message : ')
  ints = predict_class(message)
  print(ints)

  if ints[0]['intent'] == 'crop':
    
    res = get_crop_data(message)
  else:
    res = get_response(ints, intents)

  print("Output Message : ", res, "\n")

Input Message : What is the price of Mustard
[{'intent': 'crop', 'probability': '0.91031617'}]
message words :  ['What', 'is', 'the', 'price', 'of', 'Mustard']
crops :  ['Bhindi(Ladies Finger)', 'Green Chilli', 'Onion', 'Potato', 'Sponge gourd', 'Tomato', 'Bitter gourd', 'Pineapple', 'Pomegranate', 'Bottle gourd', 'Cauliflower', 'Cucumbar(Kheera)', 'Ginger(Green)', 'Lemon', 'Mango', 'Mousambi(Sweet Lime)', 'Peas Wet', 'Pumpkin', 'Mahua', 'Brinjal', 'Cabbage', 'Coriander(Leaves)', 'Guar', 'Arhar (Tur/Red Gram)(Whole)', 'Bajra(Pearl Millet/Cumbu)', 'Castor Seed', 'Cotton', 'Jowar(Sorghum)', 'Maize', 'Rice', 'Wheat', 'Banana - Green', 'Beetroot', 'Capsicum', 'Carrot', 'Cluster beans', 'Cowpea(Veg)', 'Drumstick', 'Elephant Yam (Suran)', 'French Beans (Frasbean)', 'Groundnut pods (raw)', 'Indian Beans (Seam)', 'Little gourd (Kundru)', 'Mango (Raw-Ripe)', 'Papaya (Raw)', 'Peas cod', 'Pegeon Pea (Arhar Fali)', 'Pointed gourd (Parval)', 'Ridge gourd(Tori)', 'Surat Beans (Papadi)', 'Sweet Potat

KeyboardInterrupt: ignored